In [1]:
from datasets import load_dataset
ds = load_dataset("Idavidrein/gpqa", "gpqa_diamond", split="train")

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "output", device_map="auto", load_in_4bit=True
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", padding_side="left")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [3]:
def llm(prompt):
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=2048,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.1,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

print(llm("""<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Whats a triangle<|im_end|>
<|im_start|>assistant
"""))

/home/alkin/.local/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Whats a triangle<|im_end|>
<|im_start|>assistant
A triangle is a geometric shape with three sides and three angles. It's the simplest polygon and one of the most common shapes found in nature, such as leaves or flower petals. Triangles have unique properties that make them interesting mathematical objects: they can be classified into different types based on their side lengths and angle measures. There are six main categories:

1. Equilateral triangles: All sides and angles are equal.
2. Isosceles triangles: Two sides are equal.
3. Scalene triangles: All sides and angles are different.

Each type of triangle has its own set of characteristics and uses in real-world applications, from architecture to engineering. Additionally, triangles play a crucial role in many mathematical theories, including geometry and trigonometry.<|im_end|>


In [4]:
prompt_template = """<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Answer the following multiple choice question:

{question}

A. {a}
B. {b}
C. {c}
D. {d}

Write your response in the exact following format:

```
Reason for your answer here.
...

Choice: (X)
```

You will put the correct choice in the `Chose: (_)` part, like:

`Choice: (A)`
`Choice: (B)`
`Choice: (C)`
`Choice: (D)`

Good luck!<|im_end|>
<|im_start|>assistant
I see, lets reason about this problem and finally end my message with the correct answer with the format: `Choice: (X)`:"""

In [ ]:
from tqdm.auto import tqdm
from random import shuffle

correct_num = 0
total_num = 0

for example in tqdm(ds):
    question = example["Question"].strip()
    correct_answer = example["Correct Answer"].strip()
    answers = [example["Correct Answer"].strip(), example["Incorrect Answer 1"].strip(), example["Incorrect Answer 2"].strip(), example["Incorrect Answer 3"].strip()]
    correct_answer_letter = "ABCD"[answers.index(correct_answer)]
    shuffle(answers)

    try:
        llm_answer = llm(prompt_template.format(question=question, a=answers[0], b=answers[1], c=answers[2], d=answers[3])).replace(prompt_template.format(question=question, a=answers[0], b=answers[1], c=answers[2], d=answers[3]), "").strip()

        if "Choice: (" + correct_answer_letter + ")" in llm_answer.splitlines()[-1]:
            correct_num += 1
        total_num += 1

        score = correct_num / total_num * 100

    except:
        pass

score = correct_num / total_num * 100
score

  0%|          | 0/198 [00:00<?, ?it/s]

The lifetime of a quantum state is given by the equation:
\[ \tau = \frac{1}{\pi \lambda} \]
where \(\lambda\) is the wavelength of light corresponding to the energy difference between the two states.

Given:
- \( \tau_1 = 10^{-9} \text{sec} \)
- \( \tau_2 = 10^{-8} \text{sec} \)

We need to find an energy difference \(\Delta E\) such that:
\[ \frac{1}{\pi \lambda_{1}} = \frac{1}{\pi \lambda_{2}} \]

This implies:
\[ \lambda_{1} = \lambda_{2} \]

Now, relating the wavelengths to the energy differences:
\[ \frac{\lambda_{1}}{\lambda_{2}} = \frac{E_2 - E_1}{E_1 - E_2} \]

Substituting the known values:
\[ \frac{\lambda_{1}}{\lambda_{2}} = \frac{10^{-8} \text{m}}{10^{-11} \text{m}} = \frac{10^3 \text{eV}}{10^2 \text{eV}} \]

Simplifying:
\[ \frac{10^3}{10^2} = 10^1 = 10 \]

Thus, we determine that the energy difference must be:
\[ \Delta E = 10 \text{eV} \]

Therefore, the correct option is:
`Choice: (C) 10^-8 eV`.<|im_end|>
############################################################
0.0

14.720812182741117